# Matrix completion and recommender systems


[MovieLens](movielens.umn.edu) data sets were collected by the [GroupLens Research Project](http://www.grouplens.org/) at the University of Minnesota.

This data set consists of:

- 100000 ratings (1-5) from 943 users on 1682 movies.
- Each user has rated at least 20 movies.

The `movielens.csv` file contains the full dataset. Users and items are numbered consecutively from 1. The data is randomly ordered. This is a tab separated list of

```
user id | item id | rating | timestamp
```


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.stats import pearsonr

Read the dataset from the `movielens.csv` file.


In [ ]:
dataset = pd.read_csv(
    "./movielens.csv",
    sep="\t",
    header=None,
    names=["user_id", "item_id", "rating", "timestamp"],
)
dataset

How many movies? How many people? How many ratings?


In [ ]:
n_people = np.unique(dataset.user_id).size
n_movies = np.unique(dataset.item_id).size
n_ratings = len(dataset)

print(f"{n_people} people")
print(f"{n_movies} movies")
print(f"{n_ratings} ratings")

Shuffle the data (see the function [`np.random.shuffle`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.shuffle.html)).


In [ ]:
np.random.seed(1)  # for reproducibility

idxs = np.arange(n_ratings)
np.random.shuffle(idxs)
rows_dupes = dataset.user_id[idxs]
cols_dupes = dataset.item_id[idxs]
vals = dataset.rating[idxs]

Remove unused `user_id` or `movie_id` from `rows` and `cols`. E.g., if we have a dataset like

```
user id | item id | rating 
0       |      1  |     4  
0       |      2  |     5  
3       |      2  |     5  
```
Then we would have two empty rows in the matrix (for user 1 and 2) and one empty column (for movie 0).
```
A = [
  [0 4 5],
  [0 0 0],
  [0 0 0],
  [0 ? 5],
]
```
Thus, we want to remove empty intervals.


In [ ]:
_, rows = np.unique(rows_dupes, return_inverse=True)
_, cols = np.unique(cols_dupes, return_inverse=True)

print(rows.min(), rows.max(), n_people)
print(cols.min(), cols.max(), n_movies)

Split the dataset into a subset of 80000 training ratings and 20000 testing ratings.


In [ ]:
training_data = int(0.8 * n_ratings) # use 80% of the data as training

rows_train = rows[:training_data]
cols_train = cols[:training_data]
vals_train = vals[:training_data]
rows_test = rows[training_data:]
cols_test = cols[training_data:]
vals_test = vals[training_data:]

Let us denote by $\Omega$ the set of pairs $(i,j)$ such that rating of the $i$-th user on the $j$-th movie is available in the training set (similarly, $\Omega_{\text{test}}$ is the set of testing pairs).
Let us denote by $r_{ij}$ the corresponding rating.

Create a full matrix $X \in \mathbb{R}^{n \times p}$, such that:

$$
X_{i,j} =
\begin{cases}
r_{ij} & \text{if } (i,j) \in \Omega\\
0& \text{otherwise}
\end{cases}
$$


In [ ]:
X_sparse = csr_matrix((vals_train, (rows_train, cols_train)), shape=(n_people, n_movies))
X_full = X_sparse.toarray()

## Trivial recommender system

Create a trivial recommender system, based on the average rating of each user:

$$
r^{\text{pred}}_{ij} = \frac{1}{N_i} \sum_{j : (i,j) \in \Omega} r_{ij}
$$

where $N_i = card(j : (i,j) \in \Omega)$.

Then compute the RMSE (root mean square error):

$$
\text{RMSE} = \sqrt{\frac{1}{card(\Omega_{\text{test}})} \sum_{(i,j) \in \Omega_{\text{test}}} (r_{ij} - r^{\text{pred}}_{ij})^2}
$$

and the Pearson correlation coefficient $\rho$ (use the function [scipy.stats.pearsonr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html)):

$$
\rho =
\frac
{
    \displaystyle\sum_{(i,j) \in \Omega_{\text{test}}}
       (r_{ij} - \overline{r})
       (r^{\text{pred}}_{ij} - \overline{r}^{\text{pred}})
}
{\sqrt{
    \displaystyle\sum_{(i,j) \in \Omega_{\text{test}}}
       (r_{ij} - \overline{r})^2
       }
\sqrt{
    \displaystyle\sum_{(i,j) \in \Omega_{\text{test}}}
       (r^{\text{pred}}_{ij} - \overline{r}^{\text{pred}})^2
       }}
$$

where

$$
\begin{split}
\overline{r} &= \frac{1}{card(\Omega_{\text{test}})} \sum_{(i,j) \in \Omega_{\text{test}}}
       r_{ij}
\\
\overline{r}^{\text{pred}} &= \frac{1}{card(\Omega_{\text{test}})} \sum_{(i,j) \in \Omega_{\text{test}}}
       r^{\text{pred}}_{ij}
\end{split}
$$


In [ ]:
# compute trivial predictor on the training dataset
# FILL HERE

errors_trivial = vals_test - vals_trivial

# print the metrics
RMSE_trivial = np.sqrt(np.mean(errors_trivial**2))
rho_trivial = pearsonr(vals_test, vals_trivial)[0]
print(f"RMSE: {RMSE_trivial:1.3f}")
print(f"rho : {rho_trivial:1.3f}")

# Singular value truncation (SVT) based recommender system


Implement the SVT algorithm to predict the ratings of the testing set. Set a maximum number of iterations equal to 100. Print the RMSE and $\rho$ at each iteration.

Try to calibrate the threshold to get better results.

Sketch of the SVT steps:
1. $U \Sigma V^T = A$
2. $\displaystyle A = \sum_{k : \sigma_k < \texttt{threshold}} \sigma_k u_k v_k$
3. $A_{ij} = X_{ij}$ for all $i, j$ in the training dataset
4. Compute the difference w.r.t. the $A$ at the previous step
5. Compute and save the test metrics


In [ ]:
n_max_iter = 100
threshold = 100.0
increment_tol = 1e-6

RMSE_list = list()
rho_list = list()

A = X_full.copy()

print("Iter | Increment |  RMSE |  Corr ")
for i in range(n_max_iter):
# FILL HERE

    print(f"{i+1:04} | {increment:.3e} | {RMSE_list[-1]:1.3f} | {rho_list[-1]:1.3f}")
    if increment < increment_tol:
        break

Plot the history of RMSE and $\rho$ and confront it with the trivial predictor

In [ ]:
# FILL HERE